In [ ]:
# 7. Training (same as before)

import torch
from peft import LoraConfig
from transformers import TrainingArguments
from llava.model.language_model.llava_llama import LlavaLlamaForCausalLM
from transformers import AutoTokenizer

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

training_args = TrainingArguments(
    output_dir="./llava_lora_output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,
    fp16=True,
    remove_unused_columns=False,
    report_to="none"
)

model_name = "liuhaotian/llava-vicuna-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlavaLlamaForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

from peft import get_peft_model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Load your converted dataset
with open(output_path, 'r') as f:
    train_data = json.load(f)

dataset = {"train": train_data, "validation": train_data[:100]}  # simple val split

from llava.train.train import train

train(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    training_args=training_args
)

# Save LoRA adapter
model.save_pretrained("/content/drive/MyDrive/llava-lora-spiqa")
tokenizer.save_pretrained("/content/drive/MyDrive/llava-lora-spiqa")